In [38]:
import os

ROOT = "/content/drive/MyDrive/NLP/CRM"
os.chdir(ROOT)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install tensorflow

  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached absl_py-1.0.0-py3-none-any.whl (126 kB)
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached rsa-4.8-py3-none-any.whl (39 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Using cached oauthlib-3.2.0-py3-none-any.whl (151 kB)
  Attempting uninstall: import

In [3]:
import pickle
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical 

pd.set_option('max_columns',None)

In [4]:
def load_file(name):
    """
    Function to load a pickle object
    """
    return pickle.load(open(name, "rb"))

In [5]:
ls

 Volume in drive D is Data
 Volume Serial Number is 64F6-AB9B

 Directory of D:\BePec\CRM

28-05-2022  10:15    <DIR>          .
28-05-2022  10:15    <DIR>          ..
28-05-2022  10:15    <DIR>          .ipynb_checkpoints
20-05-2022  18:15           158,802 1000_leads.xlsx
28-05-2022  09:59           195,899 beforeDL
28-05-2022  10:15           236,345 CMR_Text_Classification_DL.ipynb
27-05-2022  18:26        28,234,079 cntRandomGS.pickle
28-05-2022  09:48         4,586,225 CRM.ipynb
25-05-2022  14:40           246,531 custom_stopword_removal
25-05-2022  14:24           249,144 expand_contractions
25-05-2022  14:39           266,524 expand_shortend
27-05-2022  10:49           254,866 final_cleaned
27-05-2022  19:08           329,542 final_dataframe
25-05-2022  15:19           239,772 lemmatize
25-05-2022  14:08           233,678 remove_non_English_words
25-05-2022  14:40           266,240 spelling_correction
25-05-2022  14:24           248,313 upto_digits_removal
25-05-2022  14:39    

In [6]:
def save_file(name, obj):
    """
    Function to save an object as pickle file
    """
    with open(name, 'wb') as f:
        pickle.dump(obj, f)

In [7]:
#df = load_file('final_dataframe')

df = pd.read_excel("1000_leads.xlsx")
df = df[df.filter(regex='^(?!Unnamed)').columns]

In [8]:
df = df.rename(columns=lambda x: x.lower().replace(' ',''))
df

,leadname,location,status,statusinformation
0,Raja,hyderabad,Not Converted,"14/8/prema: share me details, available in evn..."
1,Anirudh Reddy,pune,Not Converted,"14/8/prema: cal me tmrw, shared details to ema..."
2,Sapna Dewani,bangalore,Converted,16|AuG|moHan:rnr
3,suresh,mumbai,Not Converted,14/8/17(Surendra):i want only Server 16|AuG|mo...
4,Akshay Shinde,hyderabad,Not Converted,"14/8/prema:rnr 16/8/prema: gave info, he said ..."
...,...,...,...,...
996,vipin,bangalore,Not Converted,25/4/17(Surendra):please send me details tomor...
997,dheeraj,chennai,Not Converted,"24/4/17(prema): need CT, but our venue is far...."
998,kuldeep singh,bangalore,Not Converted,24/Apr/moHan:intrstd in evng batch trail perd ...
999,ankur sharma,bangalore,Not Converted,1/5/17(Surendra):please share me details 11/5/...


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   leadname           1001 non-null   object
 1   location           980 non-null    object
 2   status             998 non-null    object
 3   statusinformation  977 non-null    object
dtypes: object(4)
memory usage: 31.4+ KB


In [10]:
df.dropna(axis=0,inplace=True)
df = df.reset_index(drop=True)
df.shape

(956, 4)

In [11]:
df['location'] = df['location'].apply(str.lower)
df['location'] = df['location'].apply(lambda x: x.strip())
df['status'] = df['status'].apply(str.lower)
df['status'] = df['status'].apply(lambda x: x.strip())
df['status'] = df['status'].replace('conveted','converted')
df['status'] = df['status'].replace('converted',1)
df['status'] = df['status'].replace('not converted',0)

In [12]:
df_loc = pd.get_dummies(df['location'],prefix='loc_')
df_loc

,loc__ahmedabad,loc__aurangabad,loc__australia,loc__bangalore,loc__bihar,loc__bilgi,loc__chennai,loc__coimbatore,loc__delhi,loc__faridabad,loc__ghazibad,loc__gujarat,loc__guntur,loc__gurgaon,loc__gurgoan,loc__hubli,loc__hyderabad,loc__india,loc__jaipur,loc__jalandhar,loc__kadapa,loc__kerala,loc__khammam,loc__kochi,loc__kolkatta,loc__madurai,loc__meerut,loc__mumbai,loc__mysore,loc__nagpur,loc__nasik,loc__nepal,loc__noida,loc__ongole,loc__online,loc__pune,loc__rajamundry,loc__rayagada,loc__solapur,loc__thane,loc__tiruttani,loc__uae,loc__usa,loc__vijayawada,loc__vishakapatnam
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
952,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
953,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
954,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### One hot encoded categorical data and cancatanated with text data


In [13]:
df_CMR = pd.concat([df[['statusinformation','status']],df_loc],axis=1)

In [14]:
df_CMR

,statusinformation,status,loc__ahmedabad,loc__aurangabad,loc__australia,loc__bangalore,loc__bihar,loc__bilgi,loc__chennai,loc__coimbatore,loc__delhi,loc__faridabad,loc__ghazibad,loc__gujarat,loc__guntur,loc__gurgaon,loc__gurgoan,loc__hubli,loc__hyderabad,loc__india,loc__jaipur,loc__jalandhar,loc__kadapa,loc__kerala,loc__khammam,loc__kochi,loc__kolkatta,loc__madurai,loc__meerut,loc__mumbai,loc__mysore,loc__nagpur,loc__nasik,loc__nepal,loc__noida,loc__ongole,loc__online,loc__pune,loc__rajamundry,loc__rayagada,loc__solapur,loc__thane,loc__tiruttani,loc__uae,loc__usa,loc__vijayawada,loc__vishakapatnam
0,"14/8/prema: share me details, available in evn...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"14/8/prema: cal me tmrw, shared details to ema...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,16|AuG|moHan:rnr,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,14/8/17(Surendra):i want only Server 16|AuG|mo...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"14/8/prema:rnr 16/8/prema: gave info, he said ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951,25/4/17(Surendra):please send me details tomor...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
952,"24/4/17(prema): need CT, but our venue is far....",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
953,24/Apr/moHan:intrstd in evng batch trail perd ...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
954,1/5/17(Surendra):please share me details 11/5/...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
save_file("beforeDL",df_CMR)

In [16]:
X = df_CMR.drop('status',axis=1)
y = df_CMR['status']

In [17]:
X

,statusinformation,loc__ahmedabad,loc__aurangabad,loc__australia,loc__bangalore,loc__bihar,loc__bilgi,loc__chennai,loc__coimbatore,loc__delhi,loc__faridabad,loc__ghazibad,loc__gujarat,loc__guntur,loc__gurgaon,loc__gurgoan,loc__hubli,loc__hyderabad,loc__india,loc__jaipur,loc__jalandhar,loc__kadapa,loc__kerala,loc__khammam,loc__kochi,loc__kolkatta,loc__madurai,loc__meerut,loc__mumbai,loc__mysore,loc__nagpur,loc__nasik,loc__nepal,loc__noida,loc__ongole,loc__online,loc__pune,loc__rajamundry,loc__rayagada,loc__solapur,loc__thane,loc__tiruttani,loc__uae,loc__usa,loc__vijayawada,loc__vishakapatnam
0,"14/8/prema: share me details, available in evn...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"14/8/prema: cal me tmrw, shared details to ema...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,16|AuG|moHan:rnr,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,14/8/17(Surendra):i want only Server 16|AuG|mo...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"14/8/prema:rnr 16/8/prema: gave info, he said ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951,25/4/17(Surendra):please send me details tomor...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
952,"24/4/17(prema): need CT, but our venue is far....",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
953,24/Apr/moHan:intrstd in evng batch trail perd ...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
954,1/5/17(Surendra):please share me details 11/5/...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
from sklearn.model_selection import train_test_split

In [19]:
#Split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,test_size=0.20, random_state=42)

# convet target column to one hot encoding - > 
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [20]:
y_train

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

# Preprocess Text data

In [21]:
!pip install wordsegment
!pip install contractions

In [22]:
import nltk
import re
import string
from wordsegment import load, segment
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from textblob import TextBlob
import contractions
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [23]:

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bhanu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
tech = ['tf','ios','iot','itil','java','mba',"mongo","ms",'msc','oracle','phd','pmp','qa','qc',"rprograming",'python','tableau','sas',"spark",'sql','tensorflow','excelr']


In [25]:
def lower_casing_text(text):
    
    """
    The function will convert text into lower case.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
         value: text in lowercase
         
    Example:
    Input : The World is Full of Surprises!
    Output : the world is full of surprises!
    
    """
    # Convert text to lower case
    # lower() - It converts all upperase letter of given string to lowercase.
    text = text.lower()
    return text

In [26]:
def remove_puctuation(text):
    """
        The function will replace punctuation with a space.

        arguments:
             input_text: "text" of type "String".

        return:
             value: text with no punctuations

        Example:
        Input : The World is Full of Pan%tu#at!io#n@s!
        Output : the world is full of Pantuations

        """
    text = text.replace(':',' ')  
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    return text

In [27]:
def remove_digits_with_words(text):
    
  
    """
    The function will remove digits with traiing or preceedign text.

    arguments:
        input_text: "text" of type "String".

    return:
        value: text with no digits with texts

    Example:
    Input : The World is Full of num234num, 9am
    Output : the world is full of 

    """

    text = re.sub('\w*\d\w*', '', text) #digits with trailing or preceeding text
    return text

In [28]:
SHORTEND_MAP = {    

"abt"           : "about",
"addres"        : "address",
"adress"        : "address",
"adrs"          : "address",
"afterwardds"   : "afterwords",
"aftr"          : "after",
"aftrnun"       : "afternoon",
"alrdy"         : "already",
"alredy"        : "already",
"alrady"        : "already",
"altdy"         : "alternet day",
"amt"           : "amount",
"anothr"        : "another",
"anthr"         : "another",
"appt"          : "appointment",
"area"          : "are",
"askng"         : "asking",
"ateend"        : "attend",
"atend"         : "attend",
"atended"       : "attended",
"atnd"          : "attend",
"atnded"        : "attended",
"atteded"       : "attended",
"attedn"        : "attend",
"attednded"     : "attended",
"attendind"     : "attending",
"attende"       : "attend",
"attnd"         : "attend",
"attnde"        : "attend",
"attnded"       : "attended",
"attndng"       : "attending",
"attned"        : "attend",
"attnede"       : "attend",
"attneded"      : "attended",
"attrended"     : "attended",
"atttend"       : "attend",
"aug"           : "august",
"auugust"       : "august",
"ad"            : "and",
"availbe"       : "available",
"avalable"      : "available",

"banglr"        : "bangalore",
"blr"           : "bangalore",
'blw'           : "bangalore",
"bangaluru"     : "bangalore",
"banglore"      : "bangalore",
"mg"            : "mahathma gandhi",
"hosp"          : "hospital",
"bnglr"         : "bangalore",
"bnlgr"         : "bangalore",
"bacth"         : "batch",
"bat"           : "batch",
"bsy"           : "busy",
"btwn"          : "between",
"b"             : "be",

"cant"          : "can not",
"cert"          : "certificate",
"cehck"         : "check",
"checkd"        : "checked",
"chck"          : "check",
"chek"          : "chek",
"chekc"         : "check",
"cal"           : "call",
"cald"          : "called",
"cacl"          : "call",
"claaroom"      : "classroom",
"clas"          : "class",
"cls"           : "classroom",
"clsroom"       : "classroom",
"CT"            : "classroom training",
"clsses"        : "classes",
"clsdemo"       : "class demo",
"clsrom"        : "classroom",
"come"        : "come",
"cmng"        : "coming",
"cmpny"       : "company",
"cncting"     : "connecting",
"cnectng"     : "connecting",
"cnnctng"     : "connecting",
"cnnecting"   : "connecting",
"cntcd"       : "contacted",
"cnfrm"       : "confirm",
"cntry"       : "contry",
"curse"       : "course",
"congirm"     : "confirm",

"dat"         : "that",
"dbs"         : "database",
"dec"         : "december",
"deffintly"   : "definitely",
"demp"        : "demo",
"dmo"         : "demo",
"den"         : "then",
"devps"       : "devops",
"dfrnt"       : "different",
"diconecting" : "disconnecting",
"diconnected" : "disconnected",
"didnt"       : "did not",
"dint"        : "did not",
"dis"         : "this",
"dng"         : "doing",
"dnt"         : "do not",
"doctors"     : "docker",
"donno"       : "do not know",
"drope"       : "drop",
"drpd"        : "dropped",
"drvg"        : "driving",
"dsmldl"      : "datascience machine learning deep learning",
"ds"          : "datascience",
"dsnt"        : "does not",
"dys"         : "days",
"dont"        : "do not",
"discus"      : "discuss",


"eachble"     : "reachable",
"eclr"        : "clear",
"emis"        : "mothly installments",
"eng"         : "engage",
"engd"        : "engaged",
"enrld"       : "enrolled",
"enrl"        : "enroll",
"even"        : "evening",
"eveng"       : "evening",
"evng"        : "evening",
"evg"         : "evening",
"exp"         : "experience",
"exler"       : "excelr",
"exlr"        : "excelr",

"fald"        : "failed",
"falg"        : "following",
"finalise"    : "finalize",
"finl"        : "final",
"finlised"    : "finalized",
"finlse"      : "finalize",
"finalised"   : "finalized",
"finlsed"     : "finalized",
"fr"          : "for",
"frnd"        : "friend",
"frnds"       : "friends",
"frshr"       : "fresher",

"gng"         : "going",
"gud"         : "good",

"hav"         : "have",
"havent"      : "have not",
"heath"       : "health",
"hom"         : "home",
"hr"          : "hour",
"hrs"         : "hours",
"hv"          : "have",
"hvng"        : "having",
"hyd"         : "hyderabad",
"hw"          : "how",


"if"          : "if",
"il"          : "i will",
"im"          : "i am",
"iam"         : "i am",
"intro"       : "introduction",
"intrrstd"    : "interested",
"intrsd"      : "interested",
"intrstd"     : "interested",
"intrtstd"    : "interested",
"inttstd"     : "interested",
"itrstd"      : "interested",
"ion"         : "ios",
"inquired"    : "enquired",
"ill"         : "i will",

"joind"       : "joined",
"joing"       : "joining",
"jul"         : "july",
"jun"         : "june",


"lkng"        : "looking",
"lokng"       : "looking",
"latr"        : "later",

"oopng"       : "looping",

"mai"         : "mail",
"mar"         : "march",
"marow"       : "tommorow",
"marrow"      : "tommorow",
"midum"       : "medium",
"mints"       : "minutes",
"minz"        : "minutes",
"mis"         : "miss",
"misd"        : "missed",
"misg"        : "missing",
"ml"          : "machine learning",
"mndy"        : "monday",
"mndytusdy"   : "monday tuesday",
"mor"         : "more",
"mrg"         : "morning",
"mrng"        : "morning",
"mndy"        : "monday",
"marag"       : "marriage",


"ned"         : "need",
"nov"         : "november",
"nt"          : "not",
"nw"          : "network",
"nyt"         : "night",
"nxt"         : "next",
"n"           : 'and',
"nd"          : "and",

"oct"         : "october",
"off"         : "office",
"offf"        : "office",
"offrd"       : "offered",
"ofc"         : "office",
"othr"        : "other",

"paln"        : "plan",
"paning"      : "planning",
"persnl"      : "personal",
"persnly"     : "personaly",
"phn"         : "phone",
"planing"     : "planning",
"plcd"        : "placed",
"plng"        : "planning",
"plz"         : "please",
"plaz"        : "please",
"plzt"        : "please",
"pm"          : "please message",
"pone"        : "phone",
"postpnd"     : "postponed",
"prblm"       : "problem",
"prev"        : "previous",
"prjt"        : "project",
"prjts"       : "project",
"provd"       : "provide",
"prvdng"      : "providing",
"psbl"        : "possible",
"pwr"         : "power",


"rt"          : "right",
"rchble"      : "reachable",
"rd"          : "road",
"r"           : "are",
"reable"      : "reachable",
"reacahble"   : "reachable",
"reachble"    : "reachable",
"reachlbe"    : "reachable",
"reahble"     : "reachable",
"rechble"     : "reachable",
"rechd"       : "reached",
"reg"         : "register",
"repnd"       : "respond",
"respd"       : "respond",
"respnd"      : "respond",
"rnrr"        : "ringing no response",
"rnr"         : "ringing no response",
"rom"         : "room",
'rprog'       : "rprogramming",

"satdy"       : "saturday",
"sat"         : "saturday",
"shred"       : "shared",
"shft"        : "shift",
"sasr"        : "sas", 
"schdle"      : "schedule",
"scheduke"    : "schedule",
"sep"         : "september",
"sept"        : "september",
"sesion"      : "session",
"shrd"        : "shared",
"shred"       : "shared",
"shw"         : "show",
"sis"         : "sister",
"sistr"       : "sister",
"soma"        : "some",
"spcl"        : "special",
"sprk"        : "spark",
"statn"       : "statistics",
"stats"       : "statistics",
"sugstd"      : "suggested",
"suprt"       : "support",
"swtchoff"    : "switched off",
"swth"        : "switch",
"swthd"       : "switch",
"swtych"      : "switch",

"twn"         : "town",
"tmrw"        : 'tomorrow',
"timg"        : "timing",
"tym"         : "time",
"tenserflow"  : "tensorflow",
"tf"          : "tensorflow",
"tim"         : "time",
"tmng"        : "timing",
"tmrws"       : "tomorrow",
"tody"        : "today",
"trang"       : "training",
"trning"      : "training",
"tues"        : "tuesday",

"wkend"       : "weekend",
"wk"          : "week",
"wednsdy"     : 'wednesday',
"weeknd"      : "weekend",
"wil"         : "will",
"wanna"       : "want to",
"wan"         : "want to",
"wana"        : "want to",
"want"        : "want to",
"wantd"       : "wanted",
"webianr"     : "webinar",
"wed"         : "wednesday",
"wen"         : "wednesday",
"wends"       : "wednesday",
"weekendeveng": "weekend evening",
"wek"         : "week",
"wk"          : "week",
"wekends"     : "weekends",
"wekknd"      : "weekend",
"weknd"       : "weekend",
"whn"         : "when",
"wid"         : "with",
"wii"         : "will",
"wkend"       : "weekend",
"wks"         : "week",
"wll"         : "will",
"wrg"         : "wrong",
"wrk"         : "work",
"wrng"        : "wrong",

"u"           : "you",
"ur"          : "you are",

"xlr"         : "Excelr",
"xcler"       : "Excelr",

"yo"          : "you",
"yr"          : "your",
"ystday"      : "yesterday",     

}


In [29]:
def expand_shortend(text,shortend_mapping = SHORTEND_MAP):
    """expand shortened words to the actual form.
       e.g. don't -> do not
            tmng  -> timing

       arguments:
            input_text: "text" of type "String".

       return:
            value: Text with expanded form of shorthened words.

       Example: 
       Input : ain't, aren't, can't, cause, can't've
       Output :  is not, are not, cannot, because, cannot have 

     """
    # tokenizeing the text into tokens
    list_of_tokens = text.split(' ')

    # check for the given token in the shortend map, if found, replaces with the mapping value

    temp_lst = []
    for word in list_of_tokens:

        # check the token is in the shortend mapping dict
        if word in SHORTEND_MAP:        
            temp_lst.append(word.replace(word,SHORTEND_MAP[word]))
        else:
            temp_lst.append(word)

    #converting list of tokens to string.
    string_of_tokens = ' '.join(str(s) for s in temp_lst)
    return string_of_tokens

In [30]:
def expand_contractions(text):
    """expand shortened words to the actual form.
       e.g. don't -> do not
            tmng  -> timing

       arguments:
            input_text: "text" of type "String".

       return:
            value: Text with expanded form of shorthened words.

       Example: 
       Input : ain't, aren't, can't, cause, can't've
       Output :  is not, are not, can not, because, cannot have """
    text = contractions.fix(text)
    return text

In [31]:
def word_segmentation(text,tech_list = tech):

    """expands concatenated words 
       e.g. moredetails -> more details
            callyou  -> call you

       arguments:
            input_text: "text" of type "String".

       return:
            value: Text with expanded form of concatenated words.

       Example: 
       Input : please sharedetails of the classtimings
       Output : please share details of the class timings

    """
    load()
    text_lst = []
    list_of_tokens = text.split(' ')
    for word in list_of_tokens:
        if word in tech_list:
            text_lst.append(word)
        else:
        #if two words are cancatenated, its seperates the words
            text_lst.extend(segment(word))
    #converting list of tokens to string.
    string_of_tokens = ' '.join(str(s) for s in text_lst)
    return string_of_tokens

In [32]:
def spelling_correction(text):
    '''
    This function will correct spellings.

    arguments:
         input_text: "text" of type "String".

    return:
        value: Text after corrected spellings.

    Example: 
    Input : This is Oberois from Dlhi who came heree to studdy.
    Output : This is Oberoi from Delhi who came here to study.

    '''
    corrected_text= str(TextBlob(text).correct())
    return corrected_text


In [33]:
CUSTOM_STOPWORDS = ['a',
 'ak',
 'al',
 'ah',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'b',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'c',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'e',
 'f',
 'ff',
 'few',
 'for',
 'from',
 'further',
 'g',
 'h',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'j',
 'just',
 'k',
 'l',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'n',
 'needn',
 "needn't",
 'nor',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'p',
 'pp'
 're',
 's',
 'same',
 'shan',
 "shan't",
 'she',
 "she's",
 'should',
 "should've",
 'shouldn',
 "shouldn't",
 'so',
 'some',
 'such',
 't',
 'than',
 'that',
 "that'll",
 'the',
 'their',
 'theirs',
 'them',
 'themselves',
 'then',
 'there',
 'these',
 'they',
 'this',
 'those',
 'through',
 'to',
 'too',
 'under',
 'until',
 'up',
 've',
 'very',
 'was',
 'wasn',
 "wasn't",
 'we',
 'were',
 'weren',
 "weren't",
 'what',
 'when',
 'where',
 'which',
 'while',
 'who',
 'whom',
 'why',
 'will',
 'with',
 'won',
 "won't",
 'wouldn',
 "wouldn't",
 'x',
 'y',
 'you',
 "you'd",
 "you'll",
 "you're",
 "you've",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'ay','di','de','eh','ex','ing','om','re','st','si','th']            

do_not_remove = ['no','not']

def custom_stopword_removal(text,cust_stopwords = CUSTOM_STOPWORDS, tech_list =tech,remove_small_tokens = True, min_len =1):
    """This function will remove stopwords which doesn't add much meaning to a sentence 
       & they can be remove safely without comprimising meaning of the sentence.

    arguments:
         input_text: "text" of type "String".

    return:
        value: Text after omitted all stopwords.

    Example: 
    Input : This is Kajal from delhi who came here to study.
    Output : ["'This", 'Kajal', 'delhi', 'came', 'study', '.', "'"]
    """
       
    
    text_lst = []
    list_of_tokens = text.split(' ')
    for word in list_of_tokens:
        # if word not in custom stopword list and not in tech
        if word not in cust_stopwords:
            # if remove small words
            if remove_small_tokens:                
                #if word length is not min lenth
                if len(word) <= min_len:
                    if word in do_not_remove or word in tech_list:
                        text_lst.append(word)                    
                else:
                    text_lst.append(word)

    return ' '.join(text_lst)


In [34]:
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()    
    text = [lemmatizer.lemmatize(word) for word in word_tokenize(text)]
    text = [lemmatizer.lemmatize(word,'v') for word in text]
    return ' '.join(text)

In [35]:
def data_cleaning(text):

    ## lowercase
    text = lower_casing_text(text)

    ## remove puctuation
    text = remove_puctuation(text)

    ## remove digits with trailing and preceeding text - sdf243abvc, 9am
    text = remove_digits_with_words(text)

    #expand contractions
    text = expand_contractions(text)

    ## word segmentations - > shareddetails -> shared details
    text = word_segmentation(text)

    ## Normalize words
    text = expand_shortend(text,shortend_mapping = SHORTEND_MAP)

    ##spelling correction
    text = spelling_correction(text)

    ## scustom stopword removal 
    text = custom_stopword_removal(text)
    
    ## Lemmetization
    text = lemmatize(text)

    ## 
    return text

In [36]:
text = '23/MaY/moHan:attnde demo 23/MaY/moHan:he want in june today check and will make token payment 30/MaY/moHan:he will research and do token payment 7/JunE/moHan:he will check tmng nd respnd in 5dys 13/JunE/moHan:buay with work some othr time 13/JulY/moHan: call me mndy 2|AuG|moHan:i will try'

In [37]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bhanu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [38]:
rxt = data_cleaning(text)

In [39]:
rxt

'att domo want june today check make token payment research token payment check tm no rest work other time call dy try'

## Prepare Textual data for train and test

In [40]:
X1_train =[]
sentence = list(X_train['statusinformation'])
for sent in sentence:
    X1_train.append(data_cleaning(sent))
X1_train

['ring no response please give money concession friend also interest ring no response not readable ring no response',
 'share detail ring no response ring no response ring no response share detail trial session share',
 'share detail tableau share previous video share morning session detail travel today attend later session ring no response unable attend due recession ring no response ring no response road',
 'not interest not call',
 'voice not audible call later ring no response come classroom please share detail busy ring no response ring no response not interest',
 'detail share ring no response ring no response ring no response ring no response',
 'cm no domo lead new camp ny finalize tide sac action call july cm no domo',
 'ring no response ring no response ring no response want classroom train share detail',
 'call later share detail morning come check tomorrow join morning join next month want classroom not interest train',
 'ring no response ring no response ring no response r

In [41]:
X1_test = []
sentence = list(X_test['statusinformation'])
for sent in sentence:
    X1_test.append(data_cleaning(sent))
X1_test

['look business analyst finance module',
 'meet ask call later detail share go detail revert ring no response ask call later ring no response call disconnect not free now check revert',
 'network issue not connect red',
 'ring no response detail share mail switch office ring no response not readable ring no response switch office ring no response',
 'ring no response mug send ring no response ring no response call not connect ring no response ring no response',
 'number service share detail mail vacation need call later confirm today want join another institute',
 'vacation need call later tuesday ring no response require domo share domo detail may not attend domo due client program share detail session share detail fee course content check course decide drive call back check course bangalore ring no response ring no response',
 'ring no response ring no response come final ed slot miss today classroom ring no response',
 'detail share att tend day session need classroom train confirm 

In [43]:
save_file('x1_train',X1_train)
save_file('x1_train',X1_test)

### convert textual input for the training and test sets into numeric form using word embeddings

In [44]:
from tensorflow.keras.layers import TextVectorization

In [49]:
tokenizer = TextVectorization(max_tokens=5000,standardize = None,)
tokenizer.adapt(X1_train)

In [50]:
tokenizer

In [52]:
tokenizer.get_vocabulary()

['',
 '[UNK]',
 'no',
 'ring',
 'response',
 'share',
 'detail',
 'call',
 'not',
 'domo',
 'need',
 'check',
 'tomorrow',
 'session',
 'want',
 'busy',
 'join',
 'back',
 'today',
 'interest',
 'classroom',
 'attend',
 'office',
 'get',
 'morning',
 'please',
 'mail',
 'try',
 'now',
 'come',
 'connect',
 'revert',
 'switch',
 'link',
 'later',
 'time',
 'month',
 'send',
 'ask',
 'weekend',
 'next',
 'know',
 'disconnect',
 'u',
 'even',
 'week',
 'end',
 'email',
 'look',
 'course',
 'say',
 'plan',
 'let',
 'ev',
 'readable',
 'already',
 'day',
 'go',
 'train',
 'confirm',
 'batch',
 'trainer',
 'room',
 'reach',
 'number',
 'inform',
 'other',
 'do',
 'available',
 'online',
 'work',
 'record',
 'present',
 'meet',
 'saturday',
 'may',
 'ch',
 'weimar',
 'sit',
 'road',
 'monday',
 'message',
 'coverage',
 'travel',
 'tide',
 'red',
 'ed',
 'dy',
 'august',
 'learn',
 'june',
 'issue',
 'due',
 'better',
 'tableau',
 'possible',
 'enrol',
 'discus',
 'register',
 'intimate',
 'in

In [53]:
tokenizer(X1_train[3])

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([ 8, 19,  8,  7], dtype=int64)>

In [54]:
X1_train[3]

'not interest not call'

In [57]:
y_train

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

AttributeError: 'list' object has no attribute 'numpy'